#Toxic Comment Classification Challenge
This [challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/overview) aims to classify the toxic comments based on many categories including whether the comments is 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'. Our model should predicts a probability of each type of toxicity for each comment. The dataset is avaiable from Kaggle website [here](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data).

In [2]:
# unzip the train and test files
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


#Libraries

In [5]:
import numpy as np
np.random.seed(42)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence

In [6]:
# read the data and directly fill the null/Nan with spaces
train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('test.csv').fillna(' ')
print(len(train))

159571


In [8]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
# print the columns of the train file
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [10]:
# the toxic categories
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [13]:
# check how many comments we have toxic and not toxic
# 0: not toxic
# 1: toxic
train['toxic'].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [44]:
# how much values we get for each catgory
print(train['severe_toxic'].value_counts())
print(train['obscene'].value_counts())
print(train['threat'].value_counts())
print(train['insult'].value_counts())
print(train['identity_hate'].value_counts())

0    157976
1      1595
Name: severe_toxic, dtype: int64
0    151122
1      8449
Name: obscene, dtype: int64
0    159093
1       478
Name: threat, dtype: int64
0    151694
1      7877
Name: insult, dtype: int64
0    158166
1      1405
Name: identity_hate, dtype: int64


In [14]:
# extract the inputs and outputs of the model for both train and test data
X_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values

In [15]:
# parameters of the Tokenizer
max_features = 30000
maxlen = 100
embed_size = 300

In [16]:
# build a tokenzier and convert the comments into numbers
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [18]:
# padd the train and test data to have same length
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
len(x_train[1])

100

In [19]:
# defind the model
def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)

    return model

model = get_model()

In [20]:
# compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
# hyper-paramters of the model
batch_size = 32
epochs = 2

In [22]:
# splite the train data to get valdation dataset
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)

In [24]:
# train the model
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                  verbose=1)


Epoch 1/2
4738/4738 [==============================] - 642s 135ms/step - loss: 0.0508 - accuracy: 0.9861 - val_loss: 0.0482 - val_accuracy: 0.9946
Epoch 2/2
4738/4738 [==============================] - 638s 135ms/step - loss: 0.0395 - accuracy: 0.9680 - val_loss: 0.0477 - val_accuracy: 0.9771


In [27]:
# Evalute the model with the test data
# and extract the probability of each type of toxicity for each comment in testset
y_pred = model.predict(x_test, batch_size=1024)
y_pred

array([[9.97604847e-01, 2.68935204e-01, 9.86119032e-01, 3.20705891e-01,
        9.40603554e-01, 1.94689125e-01],
       [9.83640275e-05, 2.16525405e-06, 4.61309101e-05, 3.33770822e-06,
        3.91968315e-05, 2.73006553e-05],
       [5.02978358e-03, 6.19175626e-05, 3.19600949e-04, 1.07015345e-04,
        1.18065428e-03, 2.96012411e-04],
       ...,
       [6.13578595e-03, 3.22805972e-05, 9.73721850e-04, 2.96285762e-05,
        1.31151546e-03, 3.77340271e-04],
       [3.11479950e-03, 2.12928644e-05, 3.81252292e-04, 9.58909586e-05,
        3.70092224e-04, 2.22219503e-03],
       [9.91191447e-01, 2.32074503e-02, 8.71221244e-01, 3.09570506e-02,
        6.38564706e-01, 1.06140804e-02]], dtype=float32)

In [34]:
# we have 153164 test comments and we predict its 6 categories
y_pred.shape

(153164, 6)

In [37]:
# extract the highest probabilty
comments_preds = np.argmax(y_pred, axis=1)

In [39]:
# find what categories our model are predict 
# it is intersting that our model doesnt predict catgory 1.
set(comments_preds)

{0, 2, 3, 4, 5}